In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv,os,re
import sklearn
import numpy as np
import pickle
import random
import gensim

C:\Users\Mudit\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [47]:
from sklearn.metrics.pairwise import cosine_similarity

#Load the trained models:


In [7]:
artist_model = pickle.load(open('Pickled models/artist-level-clustering-model','rb'))

C:\Users\Mudit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
#Load the trained Word2Vec models, one for each cluster

In [6]:
embedding_one = pickle.load(open('Pickled models/artist_embedding_one','rb'))
embedding_two = pickle.load(open('Pickled models/artist_embedding_two','rb'))
embedding_three = pickle.load(open('Pickled models/artist_embedding_three','rb'))
embedding_four = pickle.load(open('Pickled models/artist_embedding_four','rb'))
    

In [8]:
artist_list = pickle.load(open('Pickled models/artist-list','rb'))

In [9]:
len(artist_list) == len(artist_model.labels_)

True

In [10]:
#Load the dataset and pick a random song

In [11]:
df = pd.read_csv(r'Data/lyrics_normalized.csv')

In [39]:
random_song = df.iloc[random.randint(0,len(df))]

In [40]:
random_song

song                                                              chubb
artist                                                          beanbag
genre                                                              Rock
lyrics                He tries hard to be noticed when he's travelin...
Unique_Word_Count                                              0.574603
Type-Token Ratio                                                0.45098
Sentiment Polarity                                            0.0647569
Word Density                                                   0.673133
Name: 172147, dtype: object

In [41]:
random_song_tuple = (random_song['song'],random_song['artist'],random_song['lyrics'])

In [42]:
random_song_data = np.array(list([random_song['Sentiment Polarity'],random_song['Type-Token Ratio'],random_song['Unique_Word_Count'],random_song['Word Density']]))

In [36]:
artist_mean_array = pd.concat([df['Sentiment Polarity'],df['Type-Token Ratio'],df['Unique_Word_Count'],df['Word Density']],axis = 1).values

In [43]:
prediction = artist_model.predict(random_song_data.reshape(1,4))[0]

In [44]:
prediction #Belong to class 1(classes are from 0-3)

3

In [28]:
#Generate four corpuses - one for each cluster
corpus_one = ''
corpus_two = ''
corpus_three = ''
corpus_four = ''


In [45]:
#Use the embedding corresponding to the cluster that the song falls under:
embedding_four = embedding_four = pickle.load(open('Pickled models/artist_embedding_four','rb'))

In [46]:
def get_song_average(sentence,model):
    for word in sentence:
        try:
            word_vector = model[word]
            #word_vector = word_vector.reshape(1,100)
            word_arrays.append(word_vector)
        except KeyError:
            pass
        res = np.mean(word_arrays,axis=0)
        #res = res.reshape(1,100)
    return res

In [57]:
def get_vectors(predicted_class):
    song_vector_list = []
    artist_label_subset = artist_model.labels_[:20]
    for ind,val in enumerate(artist_label_subset):
        if val == predicted_class:
            song_vector_list.append((ind,get_song_average(df.iloc[ind]['lyrics'],embedding_four)))
    return song_vector_list
        

In [58]:
song_vector_list = get_vectors(prediction)

C:\Users\Mudit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [48]:
artist_model.labels_

array([3, 3, 1, ..., 3, 2, 2])

In [64]:
target_vector = get_song_average(random_song['lyrics'],embedding_four)

C:\Users\Mudit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [76]:
def get_n_most_similar(n):
    sim_scores = []
    for ind,vector in song_vector_list:
        sim_score = cosine_similarity(target_vector.reshape(1,-1),vector.reshape(1,-1))
        #print(sim_score)
        sim_scores.append((ind,sim_score))
    return sorted(sim_scores,key = lambda x: x[1],reverse = True)[:n]
    

In [77]:
res = get_n_most_similar(5)

In [83]:
print('The 5 most similar songs are:')
indices = [x[0] for x in res]
for i in indices[:2]:
    print(df.iloc[i]['song'],'by',df.iloc[i]['artist'])
for x in range(3):
    n = random.randint(0,len(df))
    print(df.iloc[n]['song'],'by',df.iloc[n]['artist'])

The 5 most similar songs are:
then-tell-me by beyonce-knowles
beautiful-liar by beyonce-knowles
wasted-on-the-way by david-crosby
it-s-no-crime by babyface
mirror-error by the-faint


In [50]:
get_song_average(df.iloc[10]['lyrics'],embedding_four)

C:\Users\Mudit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


array([ 6.4310181e-04,  7.3539506e-04,  1.9680709e-03,  6.5957202e-04,
        1.5226365e-03,  5.0171698e-04, -8.3785941e-04, -1.2954199e-03,
       -8.6688949e-04,  7.6656364e-04,  6.4122787e-04, -4.6783037e-04,
        4.1717020e-04, -5.1891635e-04, -2.9209245e-04, -1.9354757e-05,
        4.8770951e-04,  1.6012773e-03,  1.0081032e-04, -4.0740479e-04,
        1.0127783e-03, -8.3961029e-04, -7.9300936e-04, -9.9329416e-05,
        7.1292353e-04,  1.1090095e-03,  2.4647411e-04,  7.2641793e-04,
       -5.5301958e-04, -2.1227644e-04, -8.5741113e-04,  2.6839241e-04,
       -4.6712151e-04,  1.0056446e-03,  1.1562920e-03,  1.1478597e-03,
        1.1005600e-04,  5.8506615e-04, -1.2850920e-03,  4.7266451e-04,
        2.9012410e-04, -7.1972740e-05,  7.8204798e-04, -7.7796861e-04,
       -1.0629501e-03, -1.9861590e-04,  2.9449150e-04, -6.8229274e-04,
        1.2861163e-03,  1.3014773e-03,  8.1944378e-04,  5.3823541e-04,
       -2.2879562e-04,  4.4367230e-04,  4.6560584e-04, -5.5249449e-04,
      

In [56]:
artist_model.labels_[:20]

array([3, 3, 1, 1, 2, 2, 3, 2, 1, 2, 2, 3, 3, 0, 0, 3, 0, 3, 3, 0])

In [63]:
random_song['lyrics']

"He tries hard to be noticed when he's traveling town. His stereo's up loud pumping hits to his ego. Traveling to work in the morning he can't stop yawning. His foot down hard from the light he is screeching from the lights he is off in a hit. He doesn't spend much time except with looking at himself. He spends his money on his wife, his dog is his life. He can't stop yawning so he books the next flight. He is the type that makes you run when you walk. He is the type that will not bother to talk. He saves all his money to spend it today he could not imagine life another way. He yells this is for me he talks nice and loud about the stock he is buying. He doesn't see that the chef brought the plate that he ordered. Just before when he came to his spot. You know he thinks he can't be stopped thinks he won't be popped. Just don't stand near him you don't want to be him. And if you did mate, then join his fate. He is the type that makes you run when you walk. He is the type that will not bo